# Recommender Systems Baseline

> NOTEBOOK IS INCOMPLETE, MAY NOT RUN

Notebook covers the user-based KNN and SVD recommender systems to compare against the Social Media Recommender Systems

In [32]:
import numpy as np
import pandas as pd
# from sklearn.model_selection import train_test_split
from surprise import Dataset, Reader, KNNBasic, SVD
from surprise.model_selection import cross_validate, train_test_split, LeaveOneOut

## Data loading and exploration

In [62]:
df = pd.read_csv('./data/user_repo_ratings_full.csv', 
                 names=['user_id', 'repo_id', 'rating'], 
                 skiprows=1)
df.head()

,user_id,repo_id,rating
0,0x00evil,0xAX/linux-insides,0
1,0x00evil,30-seconds/30-seconds-of-code,0
2,0x00evil,996icu/996.ICU,0
3,0x00evil,AFNetworking/AFNetworking,0
4,0x00evil,AUTOMATIC1111/stable-diffusion-webui,0


In [8]:
n_users = df.user_id.unique().shape[0]
n_repos = df.repo_id.unique().shape[0]

print(f'Total obs: {df.shape[0]}')
print(f'Total users: {n_users}')
print(f'Total repos: {n_repos}')

Total obs: 316064
Total users: 1162
Total repos: 272


## Prep data for Rec Sys

Data has already been loaded in as a sparse matrix, we collect the per repo rankings, that is those repositories with most stars (1.0) in order.

In [25]:
repo_rankings = df.groupby(['repo_id']).rating.sum().sort_values(ascending=False)
repo_rankings.head()

repo_id
twbs/bootstrap                            168
EbookFoundation/free-programming-books    147
d3/d3                                     109
nodejs/node-v0.x-archive                  104
sindresorhus/awesome                      102
Name: rating, dtype: int64

# CUSTOM RECOMMENDERS

Follower-following based recommender systems here.

## Model Evaluation

Each one of the following models serves as a baseline for comparison with social media contexts. The algorithms used are KNNBasic and SVD. Even though other methods may prove stronger, these are two algorithms that directly relate to the social media context and will therefore be easier to compare.

For both algorithms, we use default values and perform 5-fold cross validation to obtain the RMSE and MAE. 

### Ratings scale

The ratings scale is 0 if a user has not starred a given repository or 1 if they have starred a given repository. This is akin to friend recommendations and allows us to utilise open triads in the social graphs.

In order to produce cosine sims, the ratings were scaled to 1 (not starred), 2 (starred).


In [17]:
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df[['user_id', 'repo_id', 'rating']], reader)

benchmark = []

algos = [KNNBasic(), SVD()]
for algo in algos:
    results = cross_validate(algo, data, measures=['RMSE', 'MAE'], 
                             cv=5, return_train_measures=True, verbose=False)

    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


KeyboardInterrupt: 

In [89]:
# Alter the ratings? - this has to be done to compute cosine sims, the distance remains the same so it shouldnt be a problem
temp = df.copy()
temp['rating'] = temp['rating']+1

temp['user_id'] = temp['user_id'].astype('category')
temp['repo_id'] = temp['repo_id'].astype('category')

# ulabels, ulevels = pd.factorize(temp['user_id'])
# rlabels, rlevels = pd.factorize(temp['repo_id'])
temp = pd.DataFrame({
    'user_name': temp['user_id'],
    'user_id': temp['user_id'].cat.codes,
    'repo_name': temp['repo_id'],
    'repo_id': temp['repo_id'].cat.codes,
    'rating': temp['rating']
})

temp

,user_name,user_id,repo_name,repo_id,rating
0,0x00evil,0,0xAX/linux-insides,0,1
1,0x00evil,0,30-seconds/30-seconds-of-code,1,1
2,0x00evil,0,996icu/996.ICU,2,1
3,0x00evil,0,AFNetworking/AFNetworking,3,1
4,0x00evil,0,AUTOMATIC1111/stable-diffusion-webui,4,1
...,...,...,...,...,...
316059,zzragida,1161,yangshun/tech-interview-handbook,267,1
316060,zzragida,1161,yt-dlp/yt-dlp,268,1
316061,zzragida,1161,ytdl-org/youtube-dl,269,1
316062,zzragida,1161,yuzu-emu/yuzu,270,1


In [121]:
import itertools
from surprise import accuracy
from collections import defaultdict


class EData:
    def __init__(self, data, rankings):
        # Generate entire training set for evaluating properties 
        self.full_trainset = data.build_full_trainset()
        self.full_anti_testset = self.full_trainset.build_anti_testset()
        
        # Set rankings
        self.rankings = rankings
        
        # Generate a train (75) / test (25) split for measuring accuracy
        self.trainset, self.testset = train_test_split(data, test_size=0.25, random_state=1)
        
        # Create leave-one-out train/test split for eval of top-N recs
        # and we create an anti-test-set for generating predictions
        LOOCV = LeaveOneOut(n_splits=1, random_state=1)
        for train, test in LOOCV.split(data):
            self.LOOCVTrain = train
            self.LOOCVTest = test
        self.LOOCVAntiTestSet = self.LOOCVTrain.build_anti_testset()
        
        # Now compute sim matrix between users to measure diversity
        self.simAlgo = KNNBaseline(
            sim_options={'name': 'cosine', 'user_based': True}
        )
        self.simAlgo.fit(self.full_trainset)
        
        
    def get_full_trainset(self):
        return self.full_trainset
    
    def get_full_anti_testset(self):
        return self.full_anti_testset
    
    def get_trainset(self):
        return self.trainset
    
    def get_testset(self):
        return self.testset
    
    def get_LOOCV_testset(self):
        return self.LOOCVTest
    
    def get_LOOCV_trainset(self):
        return self.LOOCVTrain
    
    def get_LOOCV_anti_testset(self):
        return self.LOOCVAntiTestSet
    
    def get_rankings(self):
        return self.rankings
    
    def get_sims(self):
        return self.simAlgo

class Algorithm:
    
    def __init__(self, name, algorithm):
        self.name = name
        self.algo = algorithm
        
    def evaluate(self, eval_data):
        metrics = {}
        
        # Compute accuracy
        print("Evaluating accuracy...")
        self.algo.fit(eval_data.get_trainset())
        preds = self.algo.test(eval_data.get_testset())
        metrics['RMSE'] = Metrics.RMSE(preds)
        metrics['MAE'] = Metrics.MAE(preds)
        
        # Eval top-10 via leave-one-out
        print("Evaluating top-10 with LOOCV..")
        self.algo.fit(eval_data.get_LOOCV_trainset())
        lo_preds = self.algo.test(eval_data.get_LOOCV_testset())
            
        # Generate preds for ratings not in training
        all_preds = self.algo.test(eval_data.get_LOOCV_anti_testset())
      
        # Get top 10 recs per user
        topN_preds = Metrics.get_topN(all_preds, 10)
        
        print('Evaluating rank metrics...')
        # Hit-rate - how often a movie that the user liked was recommended
        metrics['HR'] = Metrics.hit_rate(topN_preds, lo_preds)
        # Cumulative-hit-rate
        metrics['CHR'] = Metrics.cumulative_hit_rate(topN_preds, lo_preds)

        metrics["ARHR"] = Metrics.avg_reciprocal_hit_rate(topN_preds, lo_preds)

        print('Computing recs with complete dataset...')
        self.algo.fit(eval_data.get_full_trainset())
        all_preds = self.algo.test(eval_data.get_full_anti_testset())
        topN_preds = Metrics.get_topN(all_preds, 10)
        
        metrics['Coverage'] = Metrics.user_coverage(topN_preds, eval_data.get_full_trainset().n_users)
        metrics['Diversity'] = Metrics.diversity(topN_preds, eval_data.get_sims())
        metrics['Novelty'] = Metrics.novelty(topN_preds, eval_data.get_rankings())
        
        print('Done.')
        
        return metrics
        
        
class Evaluator:
    algorithms = []
    
    def __init__(self, data, rankings):
        self.data = EData(data, rankings)
    
    def add_algorithm(self, name, algorithm):
        self.algorithms.append(Algorithm(name, algorithm))
    
    def evaluate(self):
        results = {}
        for algo in self.algorithms:
            print(f'Evaluating {algo.name}...')
            results[algo.name] = algo.evaluate(self.data)
        
        # Display results
        print()
        print("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format("Algorithm", "RMSE", "MAE", "HR", "CHR", "ARHR", "Coverage", "Diversity", "Novelty" ))

        for name, metrics in results.items():
                print("{:<10} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f}".format(
                        name, metrics["RMSE"], metrics["MAE"], metrics["HR"], metrics["CHR"],  metrics["ARHR"], metrics["Coverage"], metrics["Diversity"], metrics["Novelty"]))
     
    
    
class Metrics:
    
    def MAE(predictions):
        return accuracy.mae(predictions, verbose=False)
    
    def RMSE(predictions):
        return accuracy.rmse(predictions, verbose=False)
    
    def get_topN(predictions, n=10):
        top_n = defaultdict(list)
        for uid, iid, actual, estimated, _ in predictions:
            if (estimated > 1.0): # >1 is min rating
                top_n[int(uid)].append((int(iid), estimated))
                
        for uid, ratings in top_n.items():
            ratings.sort(key=lambda x: x[1], reverse=True)
            top_n[int(uid)] = ratings[:n]
        
        return top_n
        
    def hit_rate(top_n_preds, left_out_preds):
        """Determine the hit-rate (how good) of top-N list"""
        hits = 0
        total = 0
        for lo_pred in left_out_preds:
            lo_uid = lo_pred[0]
            lo_iid = lo_pred[1]
            
            # Check if in top 10
            is_hit = False
            for tn_iid, tn_pred in top_n_preds[int(lo_uid)]:
                if (int(lo_iid) == int(tn_iid)):
                    is_hit = True
                    break
            if is_hit:
                hits += 1
            total += 1
            
        precision = hits / total
        return precision
            
    
    def cumulative_hit_rate(topN_preds, left_out_preds):
        hits = 0
        total = 0
        for lo_uid, lo_iid, actual, estimated, _ in left_out_preds:
            if (actual == 2.0): # Only look at things the user starred
                is_hit = False
                for tn_iid, tn_pred in topN_preds[int(lo_uid)]:
                    if (int(lo_iid) == tn_iid):
                        is_hit = True
                        break
                if is_hit:
                    hits += 1
                total += 1
        precision = hits/total
        return precision
    
    def avg_reciprocal_hit_rate(topN_preds, left_out_preds):
        S = 0
        total = 0
        for lo_uid, lo_iid, actual, estimated, _ in left_out_preds:
            hit_rank = 0
            rank = 0
            for tn_iid, tn_pred in topN_preds[int(lo_uid)]:
                rank += 1
                if (int(lo_iid) == tn_iid):
                    hit_rank = rank
                    break
            if hit_rank > 0:
                S += 1.0 / hit_rank
            total += 1
        return S / total
    
    def user_coverage(topN_preds, num_users):
        # Calc the percentage of users that have at least 1 good rec
        hits = 0
        for tn_uid in topN_preds.keys():
            is_hit = False
            for tn_iid, tn_pred in topN_preds[tn_uid]:
                if tn_pred > 1.0: # Todo is 1 correct number to use??
                    is_hit = True
                    break
            if is_hit:
                hits += 1
        return hits / num_users
    
    
    # TODO: NOVELTY AND DIVERSITY ARE NOT WORKING -- most likely due to topN_preds not working??
    def diversity(topN_preds, sim_algorithm):
        n = 0
        total = 0
        mat = sim_algorithm.compute_similarities()
        for uid in topN_preds.keys():
            pairs = itertools.combinations(topN_preds[uid], 2)
            for pair in pairs:
                repo1 = pair[0][0]
                repo2 = pair[1][0]
                inner_id1 = sim_algorithm.trainset.to_inner_iid(str(repo1))
                inner_id2 = sim_algorithm.trainset.to_inner_iid(str(repo2))
                sim = mat[inner_id1][inner_id2]
                total += sim
                n += 1
        return 0 # 1 - (total / n) 
    
    def novelty(topN_preds, rankings):
        n = 0
        total = 0
        for uid in topN_preds.keys():
            for rating in topN_preds[uid]:
                iid = rating[0]
                rank = rankings[iid]
                total += rank
                n += 1
        return 0 # total / n

In [131]:
# Load and process our data
reader = Reader(rating_scale=(1, 2))
data = Dataset.load_from_df(temp[['user_id', 'repo_id', 'rating']], reader)

# setup an evaluator
evaluator = Evaluator(data, repo_rankings)

# USER-BASED KNN
user_knn = KNNBasic(sim_options={'name': 'cosine', 'user_based': True})

# SVD
# svd = SVD()

# TODO: add social network stuff
# following_triads = FollowingTriadsAlgo()
# follower_triads = FollowerTriadsAlgo()
# followee_and_follower_net = FolloweeFollowerNet(type='and')
# followee_or_follower_net = FolloweeFollowerNet(type='or')

# Add the algorithms
evaluator.add_algorithm('User KNN', user_knn)
# evaluator.add_algorithm('SVD', svd)


Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [135]:
# Run the evals
evaluator.evaluate()

Evaluating User KNN...
Evaluating accuracy...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating top-10 with LOOCV..
Computing the cosine similarity matrix...
Done computing similarity matrix.
TOP N PREDS
defaultdict(<class 'list'>, {2: [(72, 1.0750300099864645)], 9: [(237, 1.024969622268857)], 11: [(254, 1.0750785559643743)], 13: [(156, 1.0750626014709324)], 14: [(50, 1.0750862925672824)], 15: [(68, 1.0500252289323644)], 16: [(215, 1.025043873322799)], 20: [(7, 1.0500084536921939)], 21: [(22, 1.02503945158024)], 24: [(264, 1.0250330155342027)], 26: [(141, 1.024963277310204)], 29: [(30, 1.0250335411942029)], 31: [(49, 1.0250324334007288)], 32: [(3, 1.0250256969305005)], 36: [(109, 1.100066458799151)], 37: [(15, 1.024975919768464)], 39: [(196, 1.0250144750148291)], 40: [(25, 1.0250397197498564)], 41: [(226, 1.0249804501546234)], 44: [(22, 1.0250406247494748)], 49: [(83, 1.0249877022589222)], 50: [(166, 1.0999872305376168)], 53: [(143, 1.02504529030083

Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Done.

Algorithm  RMSE       MAE        HR         CHR        ARHR       Coverage   Diversity  Novelty   
User KNN   0.1684     0.0420     0.4234     0.6341     0.4234     0.0000     0.0000     0.0000    


## Evaluation

Use of multiple measures to evaluate the final system quality.